# Utilización de imágenes de diferentes sensores/plataformas no indexadas en Data Cube Chile (DCC)

Durante este notebook exploraremos dos temáticas principales:

* Leer imágenes directamente desde los buckets de s3 para imágenes no indexadas en DCC.
* Manejar imágenes de diferentes extensiones, resoluciones, y alineaciones espaciales. Esto es sólo necesario para imágenes que no están indexadas al catálogo, como las que se describen a continuación. Si un producto está indexado en DCC, al hacer la consulta la alineación se realiza de manera automática y recomendamos hacerlo de esa manera.


## Leer imágenes desde buckets de s3

Para este ejemplo, utilizaremos imágenes desde s3 que estén vinculadas al Glaciar Grey.

Para esto necesitamos conocer las rutas específicas de cada imágen en el bucket.

Leamos el csv a continuación para conocer las rutas de cada imagen.

In [ ]:
import pandas as pd
s3_images = pd.read_csv('s3_imagenes_disponibles.csv')

In [ ]:
s3_images

La columna `full_path` tiene la información que requerimos para obtener las imagenes desde s3.
Sin embargo, aprovecharemos de filtrar el DataFrame para quedarnos con el sector, producto y tipo de nuestro interés.

In [ ]:
s3_images.loc[(s3_images['sector'] == 'Glaciar Grey')]

Podemos observar que, en el bucket `s3://easido-prod-dc-data-projects/saf/` disponemos de imágenes `aerial` y `planet_scope` para el glaciar grey.

In [ ]:
s3_images.loc[(s3_images['sector'] == 'Glaciar Grey') & (s3_images['product'] == 'planet_scope') & (s3_images['type'] == 'mss')]

De la misma forma, para `planet_scope` de tipo `mss` existen 4 escenas distintas con múltiples bandas espectrales.

Carguemos una escena con todas sus bandas.

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

In [ ]:
import datacube
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from odc.ui import DcViewer
from datacube.utils import masking
from datacube.utils.rio import configure_s3_access

configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

In [ ]:
import rioxarray
from rasterio.enums import Resampling

In [ ]:
dc = datacube.Datacube(app='MSS and Multiplatform') 

In [ ]:
idx = (s3_images['sector'] == 'Glaciar Grey') & (s3_images['product'] == 'planet_scope') & (s3_images['type'] == 'mss') & (s3_images['scene'] == 'REQ_6794')
bands_name = s3_images.loc[idx]['full_path'].values

# Las bandas para que esten ordenadas de forma asendente según su longitud de onda deben tener el siguiente orden:
band_order = ["coastal", "blue", "green1", "green", "yellow", "red", "redegde", "nir"]

In [ ]:
bandsi = [rioxarray.open_rasterio(f, chunks={'x':2048, 'y':2048}).squeeze(drop=True) for f in bands_name]

# extraemos el nombre de cada banda usando los últimos caracteres, después de "_", de la lista
for i, im in enumerate(bandsi):
  bandsi[i].name = bands_name[i].split("_")[-1].split('.')[0]

# Filtramos según el orden y las bandas que definimos anteriormente:
bandsif = [f for b in band_order for f in bandsi if b == f.name]

ds = xr.merge(bandsif)

In [ ]:
ds

> Alternativamente, podemos cargar la imagen de manera lazy con Dask, como lo hace DCC pasando como argumento ´chunks´ de la misma forma que las consultas habituales que se hacen al catálogo indexado: `bandsi = [rioxarray.open_rasterio(f, chunks={'x':2048, 'y':2048}).squeeze(drop=True) for f in bands_name]`. De esta forma, podemos sacar mayor provecho a la capacida de cómputo.

Podemos añadir también la fecha de la escena siguiendo el siguiente código:

In [ ]:
date = s3_images.loc[idx]['date'].max()
ds = ds.expand_dims(time=pd.to_datetime([date], format = "%Y%m%d"))

In [ ]:
ds

In [ ]:
ds[["red", "green", "blue"]].squeeze().to_array().plot.imshow(robust=True, figsize=(12, 10), vmin=500)

¿Qué resolución tenemos?

In [ ]:
ds.rio.resolution()

¿Qué sistema de referencia de coordenadas tenemos?

In [ ]:
ds.rio.crs

Genial!

Ya es posible utilizar esta escena `planet_scope`.

***

Pero, ¿Cómo genero un cubo con otras imágenes?

In [ ]:
query = {
    "product": "s2_l2a",
    #"measurements": "",
    "y": (-51.05, -50.95), 
    "x": (-73.35, -73.15),
    "time": ("2023-12-25", "2024-01-05"),
    "output_crs": "EPSG:32718",
    "resolution": (-10, 10),
    "dask_chunks": {"time": 1, 'x':2048, 'y':2048},
    "group_by": "solar_day"
}

In [ ]:
s2_ds = dc.load(**query)

In [ ]:
s2_ds

In [ ]:
s2_ds[['red', 'green', 'blue']].to_array().plot.imshow(vmin = 100, vmax = 5000, col = 'time', col_wrap = 5)

Vamos a elegir algunas bandas espectrales que se comparten en ambas imágenes

In [ ]:
ds = ds[['coastal', 'blue', 'green', 'red', 'nir']]
s2_ds = s2_ds[['coastal', 'blue', 'green', 'red', 'nir']]

Luego vamos a reproyectar `planet_scope` debido a que queremos combinarlo con `sentinel 2`. Siendo `sentinel 2` quien tiene una menor resolución espacial.

Este código ajusta la imagen de `planet_scope` al sistema de referencia de coordenada, extensión, resolución y alineación de `sentinel 2` usando el método de remuestreo `nearest`.

In [ ]:
ps_ds = ds.rio.reproject(
    s2_ds.rio.crs,
    transform=s2_ds.rio.transform(),
    shape=s2_ds.rio.shape,
    resampling = Resampling.nearest
)

Reasignamos el atributo `spatial_ref` para que tenga el valor de EPSG.

In [ ]:
ps_ds.spatial_ref.values = np.array([ps_ds.rio.crs.to_epsg()]).reshape(())

Y unimos ambos imagenes

In [ ]:
join_ds = xr.merge([s2_ds, ps_ds])
join_ds

Listo, ¡ya tenemos ambos productos unidos!

¿Veamos los resultados?

In [ ]:
join_ds[["red", "green", "blue"]].isel(time = [0, 2]).to_array().plot.imshow(vmin = 100, vmax = 5000, col = 'time', figsize = (20,10))

In [ ]:
client.close()

cluster.close()